Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print 'Training set', train_dataset.shape, train_labels.shape
  print 'Validation set', valid_dataset.shape, valid_labels.shape
  print 'Test set', test_dataset.shape, test_labels.shape

Training set (352743, 28, 28) (352743,)
Validation set (176371, 28, 28) (176371,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (352743, 784) (352743, 10)
Validation set (176371, 784) (176371, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [5]:
def logit_with_relu(dataset, (weight1, bias1, weight2, bias2)):
      logits_1 = tf.matmul(dataset, weight1) + bias1
      logits_hidden = tf.nn.relu(logits_1)
      logits_2 = tf.matmul(logits_hidden, weight2) + bias2
      return logits_2

In [6]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_1 = tf.Variable(tf.zeros([n_hidden]))
  weights_2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  variables = (weights_1, biases_1, weights_2, biases_2)
  
  # Training computation.
  # note that 'logit_with_relu' is already defined before
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit_with_relu(tf_train_dataset,variables), tf_train_labels))
  regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) # L2-regularization. Applys to weights only.(Not bias)
  regularization_hyperparameter = 5e-4
  loss += regularization_hyperparameter * regularizers
    
  # Optimizer. Need to reduce the learning rate if you want to avoid the loss increasing to infinity
  learning_rate = 0.5
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_with_relu(tf_train_dataset, variables))
  valid_prediction = tf.nn.softmax(logit_with_relu(tf_valid_dataset, variables))
  test_prediction = tf.nn.softmax(logit_with_relu(tf_test_dataset, variables))

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 499.611
Minibatch accuracy: 10.2%
Validation accuracy: 27.1%
Minibatch loss at step 500 : 129.124
Minibatch accuracy: 77.3%
Validation accuracy: 78.3%
Minibatch loss at step 1000 : 96.7809
Minibatch accuracy: 83.6%
Validation accuracy: 82.0%
Minibatch loss at step 1500 : 75.5031
Minibatch accuracy: 79.7%
Validation accuracy: 81.3%
Minibatch loss at step 2000 : 58.4565
Minibatch accuracy: 85.9%
Validation accuracy: 81.5%
Minibatch loss at step 2500 : 44.5508
Minibatch accuracy: 80.5%
Validation accuracy: 83.4%
Minibatch loss at step 3000 : 34.2326
Minibatch accuracy: 80.5%
Validation accuracy: 84.2%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [8]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_1 = tf.Variable(tf.zeros([n_hidden]))
  weights_2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  variables = (weights_1, biases_1, weights_2, biases_2)
  
  # Training computation.
  # note that 'logit_with_relu' is already defined before
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit_with_relu(tf_train_dataset, variables), tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_with_relu(tf_train_dataset, variables))
  valid_prediction = tf.nn.softmax(logit_with_relu(tf_valid_dataset, variables))
  test_prediction = tf.nn.softmax(logit_with_relu(tf_test_dataset, variables))

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[:128, :]
    batch_labels = train_labels[:128, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 406.136
Minibatch accuracy: 6.2%
Validation accuracy: 24.5%
Minibatch loss at step 500 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 1000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 1500 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 2000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 2500 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Minibatch loss at step 3000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 65.3%
Test accuracy: 71.6%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [10]:
def logit_with_relu_and_dropout(dataset, (weight1, bias1, weight2, bias2)):
      logits_1 = tf.matmul(dataset, weight1) + bias1
      logits_1 = tf.nn.dropout(logits_1, 0.5) # dropout
      logits_hidden = tf.nn.relu(logits_1) #relu
      logits_2 = tf.matmul(logits_hidden, weight2) + bias2
      logits_2 = tf.nn.dropout(logits_2, 0.5) # dropout
      return logits_2

In [11]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_1 = tf.Variable(tf.zeros([n_hidden]))
  weights_2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  variables = (weights_1, biases_1, weights_2, biases_2)
  
  # Training computation.
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
            logit_with_relu_and_dropout(tf_train_dataset, variables), tf_train_labels))
  
  # Optimizer.
  # learning rate has changed to avoid the cost from skyrocketting
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_with_relu(tf_train_dataset, variables))
  valid_prediction = tf.nn.softmax(logit_with_relu(tf_valid_dataset, variables))
  test_prediction = tf.nn.softmax(logit_with_relu(tf_test_dataset, variables))

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[:128, :]
    batch_labels = train_labels[:128, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 749.63
Minibatch accuracy: 3.1%
Validation accuracy: 18.3%
Minibatch loss at step 500 : 0.723374
Minibatch accuracy: 100.0%
Validation accuracy: 67.6%
Minibatch loss at step 1000 : 1.42012
Minibatch accuracy: 100.0%
Validation accuracy: 68.1%
Minibatch loss at step 1500 : 1.14573
Minibatch accuracy: 100.0%
Validation accuracy: 68.2%
Minibatch loss at step 2000 : 0.909578
Minibatch accuracy: 100.0%
Validation accuracy: 68.0%
Minibatch loss at step 2500 : 0.774515
Minibatch accuracy: 100.0%
Validation accuracy: 68.9%
Minibatch loss at step 3000 : 0.759226
Minibatch accuracy: 100.0%
Validation accuracy: 68.8%
Test accuracy: 75.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [13]:
batch_size = 128
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden]))
  biases_1 = tf.Variable(tf.zeros([n_hidden]))
  weights_2 = tf.Variable(tf.truncated_normal([n_hidden, num_labels]))
  biases_2 = tf.Variable(tf.zeros([num_labels]))
  var_list = (weights_1, biases_1, weights_2, biases_2)
  global_step = tf.Variable(0, trainable=False) # count the number of steps, which will used for the weight decay
  
  # Training computation.
  # dropout actually lowers performance for a relatively small network
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logit_with_relu_and_dropout(tf_train_dataset,var_list), tf_train_labels))
  regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) # L2-regularization. Applys to weights only.(Not bias)
  # you may gradually tune the regularization_hyperparameter also
  regularization_hyperparameter = 5e-4
  loss += regularization_hyperparameter * regularizers
    
  # Optimizer w/ learning rate decay
  learning_rate = tf.train.exponential_decay(1e-3, global_step=global_step,decay_steps=3001,decay_rate=0.99)
  # pass the global_step to the minimize() increments global_step as the session proceeds
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logit_with_relu(tf_train_dataset, var_list))
  valid_prediction = tf.nn.softmax(logit_with_relu(tf_valid_dataset, var_list))
  test_prediction = tf.nn.softmax(logit_with_relu(tf_test_dataset, var_list))

In [14]:
# Deep networks or adding things doesn't necessarily mean better predictors. You need to think about many things.
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print "Initialized"
  for step in xrange(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels)
  print "Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels)

Initialized
Minibatch loss at step 0 : 891.327
Minibatch accuracy: 17.2%
Validation accuracy: 13.8%
Minibatch loss at step 500 : 400.196
Minibatch accuracy: 63.3%
Validation accuracy: 67.4%
Minibatch loss at step 1000 : 291.625
Minibatch accuracy: 77.3%
Validation accuracy: 74.5%
Minibatch loss at step 1500 : 391.921
Minibatch accuracy: 71.1%
Validation accuracy: 76.8%
Minibatch loss at step 2000 : 294.915
Minibatch accuracy: 73.4%
Validation accuracy: 78.0%
Minibatch loss at step 2500 : 310.757
Minibatch accuracy: 80.5%
Validation accuracy: 78.8%
Minibatch loss at step 3000 : 300.944
Minibatch accuracy: 73.4%
Validation accuracy: 79.4%
Test accuracy: 86.4%
